Load the InceptionV3 model with pretrained ImageNet weights, unfreeze some top layers, and fine-tune it on a custom binary dataset (e.g., cats vs dogs).

In [19]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D

In [11]:
#Parameters
IMG_SIZE = 299
BATCH_SIZE = 32

In [13]:
#dataset directories
train_dir='./cats_and_dogs_filtered/train'
val_dir='./cats_and_dogs_filtered/validation'

In [14]:
# Load training dataset
train_ds=tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Load validation dataset
val_ds=tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [16]:
data_augmentation=Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [17]:
#Preprocessing layer for InterceptionV3
preprocess_layer=layers.Rescaling(1./127.5,offset=-1)
# Prepare datasets (efficient pipeline)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (preprocess_layer(x), y)).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (preprocess_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Load InceptionV3 base mode
base_model=InceptionV3(weights='imagenet',include_top=False,input_shape=(IMG_SIZE,IMG_SIZE,3))
base_model.trainable=False

87910968/87910968 [==============================] - 11s 0us/step


In [21]:
model=Sequential([
    data_augmentation,
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=3)

Epoch 1/3
63/63 [==============================] - 384s 6s/step - loss: 0.1414 - accuracy: 0.9475 - val_loss: 0.0237 - val_accuracy: 0.9940
Epoch 2/3
30/63 [=============>................] - ETA: 1:50 - loss: 0.0562 - accuracy: 0.9831

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [ ]:
# Recompile with lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=3)

In [ ]:
for images, labels in val_ds.take(1):
    pred = model.predict(images[:1])
    plt.imshow(tf.cast(images[0]*127.5+127.5, tf.uint8))
    plt.title(f"Predicted: {'Dog' if pred[0][0] > 0.5 else 'Cat'}")
    plt.axis('off')
    plt.show()
    break